# Retrieval Augmented Generation Deep Dive 

In this notebook, i'll be using Langchain, LangServe, LangGraph, LangGraph Studio and LangSmith family for building and exploring RAG from scratch and teh tehcniques we can do with Agents

### Brief History

## Introduction

### Agents

#### Prompt chaining workflow


##### LangChain PromptTemplate


##### LangChain ChatPromptTemplate


##### LangGraph StateGraph Chains

#### Context Management

##### LangChain ConversationBufferMemory

##### LangChain ConversationSummaryMemory


##### LangChain ConversationBufferWindowMemory


##### LangChain ConversationTokenBufferMemory


##### LangGraph MemorySaver


##### LangGraph MessagesState


##### LlamaIndex ChatMemoryBuffer








##### LlamaIndex VectorMemory

#### Routing Workflow



#### Parallelization Workflow



##### LangGraph Parallel Execution


##### LangChain RunnableParallel

#### Orchestrator Workers Workflow



##### LangGraph Multi-Agent Patterns


##### LangChain Agent Executors


##### LlamaIndex AgentRunner

### But Why RAG?

Talk about LLM system in general, while introducing agents, where those workflows lack the limit of llms in context and actions

### Finding the Data

#### Webscraping


##### LangChain WebBaseLoader


##### LangChain AsyncHtmlLoader


##### LangChain SitemapLoader


##### LangChain PlaywrightURLLoader


##### LlamaIndex SimpleWebPageReader



##### LlamaIndex BeautifulSoupWebReader

#### Document Loading


##### LangChain PyPDFLoader


##### LangChain UnstructuredFileLoader


##### LangChain CSVLoader


##### LangChain JSONLoader


##### LlamaIndex SimpleDirectoryReader


##### LlamaIndex PDFReader

### Preprocessing the documents

#### Splitting


##### LangChain RecursiveCharacterTextSplitter


##### LangChain TokenTextSplitter


##### LangChain MarkdownHeaderTextSplitter


##### LangChain PythonCodeTextSplitter


##### LlamaIndex SentenceSplitter


##### LlamaIndex SemanticSplitterNodeParser


##### LlamaIndex HierarchicalNodeParser

#### Chunking



##### LangChain SemanticChunker


##### LangChain ParentDocumentRetriever


##### LlamaIndex SimpleNodeParser


##### LlamaIndex SentenceWindowNodeParser

#### Embedding


##### LangChain OpenAIEmbeddings


##### LangChain HuggingFaceEmbeddings


##### LlamaIndex OpenAIEmbedding



##### LlamaIndex HuggingFaceEmbedding

### Storing Documents

#### Vector Databases


##### LangChain Chroma Integration


##### LangChain Pinecone Integration


##### LangChain FAISS Integration


##### LlamaIndex ChromaVectorStore



##### LlamaIndex PineconeVectorStore

#### Knowledge Graphs


##### LangGraph StateGraph


##### LangChain Neo4jGraph



##### LlamaIndex KnowledgeGraphIndex

#### SQL


##### LangChain SQLDatabase


##### LangChain SQLDatabaseChain



##### LlamaIndex SQLStructStoreIndex

### Retrieval Mechanisms

#### Vector search


##### LangChain VectorStoreRetriever


##### LangChain MultiVectorRetriever


##### LlamaIndex VectorIndexRetriever



##### LlamaIndex VectorIndexAutoRetriever

#### Tree Search

#### Node Search

#### Hybrid Search

##### LangChain EnsembleRetriever
##### LangChain BM25Retriever
##### LlamaIndex QueryFusionRetriever

##### LangChain ConditionalEdge


##### LangGraph Router Patterns


##### LlamaIndex RouterQueryEngine

### Evaluation

#### Faithfulness & Accuracy

#### RAGAS (RAG Assessment)



##### LangSmith + RAGAS Integration


##### LangChain Evaluation Chains

#### TruLens RAG Triad


#### Multi-Agent Metrics


#### Advanced Agentic Patterns

#### Human Evaluation


#### LLM-as-Judge



##### LangSmith Tracing


##### LangSmith Evaluation Datasets


##### LangSmith Custom Evaluators

## A Complete Agentic System



##### LangGraph Agent Architecture


##### LangChain Agent Types (ReAct, Plan-and-Execute)


##### LangSmith Agent Monitoring


##### LlamaIndex Multi-Agent Orchestrator

## Limitations & Variations

#### RAPTOR

#### Self-RAG

#### CRAG

#### Adaptive RAG

## Summary

## Citations